In [127]:
import cplex
from cplex.exceptions import CplexError
from docplex.mp.model import *
import random
import matplotlib.pyplot as plt
import docplex.mp.model as Solution
import docplex.cp.parameters as param
import numpy as np
import pandas as pd
import math as math
import csv

In [128]:
rnd = np.random
rnd.seed(0)

In [129]:
def maximum(a, b):
    if a >= b:
        return a
    else:
        return b

In [130]:
data = []
cost = []
t = [] # thời gian di chuyển từ điểm i đến điểm j

# Read the CSV file
with open('distance_matrix.csv', 'r', encoding="utf-8-sig") as file:
    csv_reader = csv.reader(file)

    # Iterate over each row in the CSV file
    for row in csv_reader:
        # Convert the elements in the row from strings to integers
        distance = [int(float(element)) for element in row]
        data.append(distance)
        time_trans = [int(float(element)) * 6 for element in row]
        t.append(time_trans)
    

Lấy các ràng buộc từ khách hàng

In [131]:
data_demands = []
ready_time = []
due_time = []
service_time = []

# Read the customer data from the text file
with open('input.txt', 'r') as file:
    # Skip the header line
    next(file)

    # Process each line in the file
    for line in file:
        # Remove leading/trailing whitespace and split the line into attributes
        attributes = line.strip().split()
        if len(attributes) >= 7:
            demand = int(attributes[3])
            redtime = int(attributes[4])
            dutime  =  int(attributes[5])
            sertime = int(attributes[6])

            # Add the dictionary to the data list
            data_demands.append(demand)
            ready_time.append(redtime)
            due_time.append(dutime)
            service_time.append(sertime)

Khai báo các chỉ số cơ bản

In [132]:
# n  = len(data) - 1 #số lượng khách hàng
n = 10
Q = 40 #sức chứa của xe
K = 2  #số lượng xe
V = [i for i in range(1,K+1)] # set of vehicle
C = [i for i in range(1,n+1)] #set of client
N = [0] + C #set of node (warehouse + client)
q = {i:data_demands[i] for i in C} #yêu cầu của khách hàng i
s = {i:service_time[i] for i in C} #thời gian phục vụ khách hàng i
a = {i:ready_time[i] for i in C} #thời gian sớm nhất phục vụ khách hàng i
b = {i:due_time[i] for i in C} #thời gian muộn nhất phục vụ khách hàng i
total_demad = sum(q.values())

In [133]:
A = [(i, j) for i in N for j in N if i != j]
S = [(i, j,k) for i in N for j in N for k in V if i != j]
c = {(i, j): data[i][j] for i, j in A} #chi phí di chuyển từ điểm i đến điểm j

In [134]:
mdl = Model('VRPTW')

In [135]:
x = mdl.binary_var_dict(S, name='x')
w = mdl.continuous_var_dict(C, name='w') #thời gian bắt đầu phục vụ 
u = mdl.continuous_var_dict(C, ub=Q, name='u') #số lượng hàng trên phương tiện

In [136]:
if total_demad/K <=Q:
    mdl.minimize(mdl.sum(c[i, j]*x[i,j,k] for i, j,k in S))
    mdl.add_constraints(mdl.sum(x[i, j,k] for j in N  for k in V if j != i) == 1 for i in C)
    mdl.add_constraints(mdl.sum(x[i, j,k] for i in N  for k in V if i != j) == 1 for j in C)
    mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j,k], mdl.sum(x[j,f,k] for f in N if f != i and f != j) == 1) for i, j,k in S)
    mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j,k], w[i] + s[i] + t[i][j] <= w[j]) for i, j,k in S if i != 0 and j != 0)
    mdl.add_constraints(w[i] <= b[i] for i in C)
    mdl.add_constraints(w[i] >= a[i] for i in C)
    mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j,k], u[i]+q[j] == u[j]) for i, j,k in S if i != 0 and j != 0)
    mdl.add_constraints(u[i] >= q[i] for i in C)
    # mdl.parameters.mip.pool.intensity.set(4)
    mdl.parameters.mip.limits.populate(4000)
    solution = mdl.solve(log_output=True)
else:
    print(f"{K} vehical with capacity {Q} not enough for optimize solution")


2 vehical with capacity 40 not enough for optimize solution


In [137]:
try :
    solution.display()
except Exception as e:
    print(e)

solution for: VRPTW
objective: 153
status: OPTIMAL_SOLUTION(2)
x_0_2_1 = 1
x_0_5_3 = 1
x_0_8_5 = 1
x_0_9_4 = 1
x_1_0_1 = 1
x_2_1_1 = 1
x_3_7_3 = 1
x_4_0_4 = 1
x_5_3_3 = 1
x_6_4_4 = 1
x_7_0_3 = 1
x_8_10_5 = 1
x_9_6_4 = 1
x_10_0_5 = 1
w_1 = 927.000
w_2 = 825.000
w_3 = 123.000
w_4 = 738.000
w_5 = 15.000
w_6 = 636.000
w_7 = 225.000
w_8 = 302.000
w_9 = 534.000
w_10 = 410.000
u_1 = 40.000
u_2 = 30.000
u_3 = 20.000
u_4 = 40.000
u_5 = 10.000
u_6 = 30.000
u_7 = 40.000
u_8 = 20.000
u_9 = 10.000
u_10 = 30.000


In [138]:
try :
    active_arcs = [a for a in S if x[a].solution_value > 0.5]
    print(active_arcs)
except Exception as e:
    print(e)

Model<VRPTW> has not been solved yet


In [139]:
def find_all_routes(data_list):
    routes = []
    pre_value = -1
    current_route = []
    while True:
        if len(data_list) == 0:
            break
        for i in data_list:
            if pre_value == 0:
                routes.append(current_route.copy())
                current_route.clear()
                pre_value = -1
                break
            if i[0] == 0 and len(current_route) == 0:
                current_route.extend([i[0], i[1]])
                pre_value = i[1]
                data_list.remove(i)
                continue
            elif i[0] == pre_value:
                current_route.append(i[1])
                pre_value = i[1]
                data_list.remove(i)
                if len(data_list) == 0:
                    routes.append(current_route)
                    break
    return routes

In [140]:
def format_route(route):
    return " -> ".join(str(node) for node in route)

In [141]:
result_routes = find_all_routes(active_arcs)
if result_routes == -1:
    print("No result")
else:
    print("solution for VRPTW")
    print(f"objective: {solution.get_objective_value()}")
    for i, route in enumerate(result_routes, 1):
        print(f"Route {i}: {format_route(route)}")


solution for VRPTW
objective: 153.0
